In [1]:
import tensorflow as tf
from transformers import T5Tokenizer, TFAutoModelForSeq2SeqLM
import gradio as gr

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
model_path = "/content/drive/MyDrive/Project/t5_tf_text_summarization"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/Project/t5_tf_text_summarization.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [4]:
def summarize(text, max_length=200, min_length=30):
    """Generate a refined summary with improved coherence."""
    input_text = f"summarize: {text}"
    inputs = tokenizer(input_text, return_tensors="tf", truncation=True, padding="max_length", max_length=512)
    summary_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=max_length,
    min_length=min_length,
    num_beams=8,  # Stronger beam search for better quality
    length_penalty=1.5,  # Balance summary length
    no_repeat_ngram_size=4,  # Prevent redundant phrases
    repetition_penalty=3.5,  # Strongly discourages repetition
    temperature=0.6,  # Low randomness for more controlled output
    early_stopping=True
)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
text = """Artificial intelligence (AI) is a branch of computer science that aims to create machines
that can perform tasks that typically require human intelligence. These tasks include learning,
reasoning, problem-solving, perception, language understanding, and decision-making. AI has various
applications, such as in healthcare, finance, robotics, and autonomous vehicles. In recent years,
deep learning and neural networks have significantly advanced the capabilities of AI systems,
enabling them to achieve superhuman performance in some tasks, such as image recognition and
natural language processing."""

summary = summarize(text)
print("Summary:", summary)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Summary: artificial intelligence is a branch of computer science that aims to create machines that can perform tasks that typically require human intelligence. AI has various applications, such as healthcare, finance, robotics and autonomous vehicles.


In [5]:
gui = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, placeholder="Enter full text here..."),
    outputs=gr.Textbox(lines=5, placeholder="Summary will appear here..."),
    title="T5 Text Summarizer",
    description="Enter a long text in the box below and click 'Submit' to get a summarized version.",
)

# Launch the app
gui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://330ec16c50471770f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated_summary)
    return scores

# Reference summary (for testing purposes, you can adjust this)
reference_summary = """AI is a field of computer science focused on creating machines that mimic human intelligence,
including learning, reasoning, problem-solving, perception, and decision-making. Recent advances in deep learning
and neural networks have enhanced AI capabilities, especially in image recognition and language processing."""

# Generated summary using the summarize function
generated_summary = summarize(text)

# Calculate ROUGE score
rouge_scores = calculate_rouge(reference_summary, generated_summary)

# Print scores
print("ROUGE-1:", rouge_scores['rouge1'].fmeasure)
print("ROUGE-2:", rouge_scores['rouge2'].fmeasure)
print("ROUGE-L:", rouge_scores['rougeL'].fmeasure)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ROUGE-1: 0.3116883116883117
ROUGE-2: 0.16
ROUGE-L: 0.3116883116883117


In [2]:
!pip install rouge-score
!pip install gradio


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=98b0255f4785f33a55ce04f4424c13250c36b27d7826513a8d37aa08214f7845
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's depende